---

<h2 style='color:darkblue; text-align:center;'><strong>Interactive TFX Pipeline for Taxi Demand Prediction</strong></h2>

<h3 style='color:darkblue;'><strong>Introduction</strong></h3>
<p style='color:black; font-size:16px; text-align:justify;'>
On our journey to improve taxi services, we've created a sophisticated <em style='color:darkgreen;'><strong>Interactive TensorFlow Extended (TFX) Pipeline</strong></em>. This pipeline, enhanced with thorough preprocessing in BigQuery, uses <em style='color:darkgreen;'><strong>Exploratory Data Analysis (EDA)</strong></em> and detailed feature engineering, focusing particularly on <em style='color:darkgreen;'><strong>hourly data granularity</strong></em>. A key feature of our pipeline is its interactivity, which allows for ongoing updates and improvements, making it suitable for deployment in real-world scenarios. We've included advanced elements like Tuner and Resolver in our pipeline, aiming to meet our <strong>business goal</strong>: improving taxi distribution in the city, thus reducing the waiting time for customers.
</p>

<h3 style='color:darkblue;'><strong>Data Preprocessing and Pipeline Components</strong></h3>
<p style='color:black; font-size:16px; text-align:justify;'>
Data preprocessing, conducted within BigQuery, forms the backbone of our pipeline. It encompasses joining taxi data with weather data, capping outliers, encoding variables, extracting time-related features, and aggregating the data for insightful analysis. This thorough preparation paves the way for the subsequent stages of our TFX pipeline. The pipeline employs a series of critical components, including:

- <strong>CsvExampleGen</strong>: Manages data input, organizing it into distinct splits for training, evaluation, and testing.
- <strong>StatisticsGen</strong> and <strong>SchemaGen</strong>: Facilitate a deep understanding of the data's distribution and structure.
- <strong>ExampleValidator</strong>: Identifies and flags any anomalies or inconsistencies in the data.
- <strong>Transform</strong>: Performs feature engineering, enhancing the data's predictive power.
- <strong>Tuner</strong>: Optimizes model hyperparameters, ensuring peak performance.
- <strong>Trainer</strong>: Develops robust predictive models.
- <strong>Evaluator</strong>: Assesses the model's effectiveness against defined metrics.
- <strong>Resolver</strong>: Manages model versions, determining the best model for deployment.
- <strong>Pusher</strong>: Deploys the approved model, making it available for real-world application.


---

<h4 style='color:darkblue;margin-left:20px;'><strong>Importing Libraries</strong></h4>


In [1]:
import os
import pandas as pd
import pprint
import shutil
import tempfile
import urllib.request
from absl import logging
from pathlib import Path

import tensorflow as tf

from tfx import v1 as tfx

# Setting up logging and pprint
# tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()
# Set TensorFlow logging to error-only
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all, 1 = no info, 2 = no info and warnings, 3 = no info, warnings, and errors
tf.get_logger().setLevel('ERROR')

2023-12-03 22:06:58.971880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 22:07:07.369244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-pac

In [2]:
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
print(f"TensorFlow Data Validation version: {tfdv.__version__}")

TensorFlow Data Validation version: 1.14.0


In [3]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [4]:
from tfx.orchestration.metadata import sqlite_metadata_connection_config

metadata_config = sqlite_metadata_connection_config('metadata.db')


In [5]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.components import StatisticsGen
from tfx.components import SchemaGen


In [6]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.13.1
TFX version: 1.14.0


<h4 style='color:darkblue;margin-left:20px;'><strong>Setting Project and Data Paths</strong></h4>


In [7]:
GOOGLE_CLOUD_PROJECT = 'mlops-363723'         
GOOGLE_CLOUD_PROJECT_NUMBER = '75674212269'  
GOOGLE_CLOUD_REGION = 'us-central1'          
GCS_BUCKET_NAME = 'chicago_taxitrips'  

PIPELINE_NAME = 'demand-prediction'
# Define the GCS_BUCKET_NAME
GCS_BUCKET_NAME = 'chicago_taxitrips'
DATA_DIRECTORY = 'gs://chicago_taxitrips/DATA_DIRECTORY'

# # Paths for users' data.
TRAIN_DATA_FILENAME = 'training_data.csv'
VALIDATION_DATA_FILENAME = 'validation_data.csv'
TEST_DATA_FILENAME = 'test_data.csv'

TRAIN_DATA_PATH = os.path.join(DATA_DIRECTORY, TRAIN_DATA_FILENAME)
VALIDATION_DATA_PATH = os.path.join(DATA_DIRECTORY, VALIDATION_DATA_FILENAME)
TEST_DATA_PATH = os.path.join(DATA_DIRECTORY, TEST_DATA_FILENAME)


# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Metadata Configuration:
metadata_config = sqlite_metadata_connection_config('metadata.db')

# # # Path for TensorBoard logs.
# TENSORBOARD_LOG_DIR = 'gs://{}/tensorboard_logs/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Define the TensorBoard log directory
TENSORBOARD_LOG_DIR = os.path.join(os.getcwd(), 'tensorboard_logs')

# Ensure the TensorBoard log directory exists
os.makedirs(TENSORBOARD_LOG_DIR, exist_ok=True)

# Now you can print or display the directory to confirm it's set
print(f"TensorBoard logs will be saved to: {TENSORBOARD_LOG_DIR}")
# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('DATA_DIRECTORY:', DATA_DIRECTORY)
print('TRAIN_DATA_PATH:', TRAIN_DATA_PATH)
print('TEST_DATA_PATH:', TEST_DATA_PATH)
print('VALIDATION_DATA_PATH:', VALIDATION_DATA_PATH)
print('PIPELINE_ROOT:', PIPELINE_ROOT)

TensorBoard logs will be saved to: /home/jupyter/tensorboard_logs
DATA_DIRECTORY: gs://chicago_taxitrips/DATA_DIRECTORY
TRAIN_DATA_PATH: gs://chicago_taxitrips/DATA_DIRECTORY/training_data.csv
TEST_DATA_PATH: gs://chicago_taxitrips/DATA_DIRECTORY/test_data.csv
VALIDATION_DATA_PATH: gs://chicago_taxitrips/DATA_DIRECTORY/validation_data.csv
PIPELINE_ROOT: gs://chicago_taxitrips/pipeline_root/demand-prediction


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>In this cell, various project and data paths are defined, including Google Cloud project details, GCS bucket names, and paths for training, validation, and test data. It also sets up paths for pipeline artifacts, modules, and model serving directories. This step is crucial for organizing and accessing data and resources efficiently.</span>


In [8]:
context = InteractiveContext()

In [ ]:
# Read the first few lines of the TRAIN_DATA_PATH
with tf.io.gfile.GFile(TRAIN_DATA_PATH, 'r') as f:
    lines = [f.readline() for _ in range(5)]

print("".join(lines))

<h4 style='color:darkblue;margin-left:20px;'><strong>CsvExampleGen</strong></h4>



In [9]:
# Define the input configuration for CsvExampleGen
input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern=TRAIN_DATA_FILENAME),
    example_gen_pb2.Input.Split(name='eval', pattern=VALIDATION_DATA_FILENAME),
    example_gen_pb2.Input.Split(name='test', pattern= TEST_DATA_FILENAME)
])



<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell sets up the input configuration for CsvExampleGen, a component of the TFX pipeline. It defines the data splits for training, evaluation, and testing, specifying the file patterns for each. This step is crucial for ensuring that the pipeline correctly processes different data subsets during training, evaluation, and testing phases.</span>


In [10]:
# Initialize CsvExampleGen with the specified input configuration
example_gen = CsvExampleGen(
    input_base=DATA_DIRECTORY,
    input_config=input_config
)
# Run the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell initializes the CsvExampleGen component with the defined input configuration and then runs it. CsvExampleGen is responsible for ingesting data, converting it to the TFX internal format, and creating appropriate splits for the pipeline.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>StatisticsGen</strong></h4>


In [11]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs["examples"]
)
context.run(statistics_gen)
context.show(statistics_gen.outputs["statistics"])

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>In this cell, the StatisticsGen component is used to generate statistics over the data. It takes the output of CsvExampleGen as input. These statistics are essential for understanding data distributions and patterns, which can inform subsequent data processing and model training steps.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>SchemaGen</strong></h4>


In [12]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs["statistics"],
    infer_feature_shape=True
)
context.run(schema_gen)
context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'day',INT,required,,-
'day_cos',FLOAT,required,,-
'day_sin',FLOAT,required,,-
'demand',INT,required,,-
'duration',FLOAT,required,,-
'hour',INT,required,,-
'hour_cos',FLOAT,required,,-
'hour_sin',FLOAT,required,,-
'month',INT,required,,-


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell utilizes the SchemaGen component to generate a schema based on the data's statistics. The schema defines the expected types, shapes, and properties of the data, and is a crucial part of ensuring data integrity and consistency throughout the pipeline.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Data Validation with ExampleValidator</strong></h4>


In [13]:

from tfx.components import ExampleValidator
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell initializes and runs the ExampleValidator component, which checks for anomalies and inconsistencies in the data against the generated schema. This step is crucial for ensuring data quality and detecting potential issues early in the pipeline.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Displaying Data Anomalies</strong></h4>


In [14]:
context.show(example_validator.outputs['anomalies'])

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>Here, the output of the ExampleValidator component is displayed, showcasing any anomalies found in the data. This visual representation helps in quickly identifying and addressing data quality issues.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Defining Taxi Constants Module</strong></h4>


In [15]:
_taxi_constants_module_file = 'taxi_constants.py'

In [16]:
%%writefile {_taxi_constants_module_file}
NUMERICAL_FEATURES = [ 'trip_total',
    'trip_miles',
    'duration',
    'temperature_2m',
    'relativehumidity_2m',
    'precipitation',
    'rain',
    'snowfall','hour_sin',
    'hour_cos',
    'day_sin',
    'day_cos',
    'month_sin',
    'month_cos']



CATEGORICAL_NUMERICAL_FEATURES = [
    'public_holiday', 'weathercode','pickup_community_area', 'year'
]


# Keys
LABEL_KEY = 'demand'

def t_name(key):
    """
    Rename the feature keys so that they don't clash with the raw keys when
    running the Evaluator component.
    Args:
    key: The original feature key
    Returns:
    key with '_xf' appended
    """
    return key + '_xf'


Overwriting taxi_constants.py


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>In this cell, a module file name for taxi constants is defined. This module will contain key constants and parameters used throughout the notebook or pipeline, providing a centralized location for such configurations.here we defined numerical and categorical features, as well as utility functions. This modular approach enhances code organization and reusability, especially for feature definitions and transformations</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Transform Module</strong></h4>



In [17]:
TRANSFORM_MODULE_PATH = "chicago_taxi_transform.py"


<h4 style='color:darkblue;margin-left:20px;'><strong>Initializing Transform Component</strong></h4>


In [29]:
%%writefile {TRANSFORM_MODULE_PATH}

import tensorflow as tf
import tensorflow_transform as tft
import taxi_constants

_NUMERICAL_FEATURES = taxi_constants.NUMERICAL_FEATURES
_CATEGORICAL_NUMERICAL_FEATURES = taxi_constants.CATEGORICAL_NUMERICAL_FEATURES
_LABEL_KEY = taxi_constants.LABEL_KEY

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor."""
    default_value = '' if x.dtype == tf.string else 0
    if not isinstance(x, tf.sparse.SparseTensor):
        return x
    return tf.squeeze(
        tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value),
        axis=1)

def z_score_normalization(inputs, feature_name):
    """Apply Z-score normalization on a feature."""
    return tft.scale_to_z_score(_fill_in_missing(inputs[feature_name]))

# def log_transformation(inputs, feature_name):
#     """Apply log transformation on a feature."""
#     feature_values = tf.where(inputs[feature_name] <= 0, tf.constant(0.01, dtype=tf.float32), inputs[feature_name])
#     return tf.math.log1p(feature_values)
def log_transformation(inputs, feature_name):
    """Apply log transformation on a feature."""
    # Ensure the input feature is in float32
    feature_values = tf.cast(inputs[feature_name], tf.float32)

    # Replace non-positive values with a small positive number
    feature_values_safe = tf.where(
        feature_values <= 0, 
        tf.constant(0.01, dtype=tf.float32),
        feature_values
    )

    return tf.math.log1p(feature_values_safe)

def square_root_transformation(inputs, feature_name):
    """Apply square root transformation on a feature."""
    feature_values = tf.where(inputs[feature_name] <= 0, tf.constant(0.01, dtype=tf.float32), inputs[feature_name])
    return tf.math.sqrt(feature_values)


def preprocessing_fn(inputs):
    """Preprocess input columns into transformed columns."""
    outputs = {}
    
    # Numerical Features: Apply Z-score normalization.
    for key in _NUMERICAL_FEATURES:
        outputs[taxi_constants.t_name(key)] = z_score_normalization(inputs, key)
    
    # Categorical Numerical Features
    for key in _CATEGORICAL_NUMERICAL_FEATURES:
        # Ensure the feature exists in the inputs
        if key in inputs:
            outputs[taxi_constants.t_name(key)] = _fill_in_missing(inputs[key])

    # Feature Transformations
    outputs['log_trip_total'] = log_transformation(inputs, 'trip_total')
    outputs['log_trip_miles'] = log_transformation(inputs, 'trip_miles')
    outputs['log_duration'] = log_transformation(inputs, 'duration')
    outputs['sqrt_precipitation'] = square_root_transformation(inputs, 'precipitation')
    
    if _LABEL_KEY in inputs:
        
        outputs[_LABEL_KEY]= log_transformation(inputs, _LABEL_KEY)
        
    return outputs

Overwriting chicago_taxi_transform.py


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>In this cell, the Transform component of the TFX pipeline is initialized. It uses the outputs from CsvExampleGen and SchemaGen, along with the transform module file created earlier, to apply preprocessing and feature engineering to the data.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Running Transform Component</strong></h4>


In [30]:
transform = tfx.components.Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_PATH),
)

In [31]:
context.run(transform, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying chicago_taxi_transform.py -> build/lib
copying taxi_transform.py -> build/lib
copying taxi_trainer.py -> build/lib
copying taxi_constants.py -> build/lib
copying taxi_tuner.py -> build/lib
installing to /tmp/tmpmfu05amv
running install
running install_lib
copying build/lib/taxi_transform.py -> /tmp/tmpmfu05amv
copying build/lib/taxi_tuner.py -> /tmp/tmpmfu05amv
copying build/lib/taxi_constants.py -> /tmp/tmpmfu05amv
copying build/lib/taxi_trainer.py -> /tmp/tmpmfu05amv
copying build/lib/chicago_taxi_transform.py -> /tmp/tmpmfu05amv
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpmfu05amv/tfx_user_code_Transform-0.0+73bae87b1f6426ef18fb06e767ae874f1b88550eac3c546056d1c8d6924d7620-py3.10.egg-info
running install_scripts
creating /tmp/tmpmfu05amv/tfx_user_code_Transform-0.0+73bae87b1f6426ef18fb06e767ae874f1b88550eac3c546056d1c8d6924d7620.dist-info/WHEEL
creating '/tmp/tmpuqkguokl/tfx_user_code_Transform-0.0+73bae87b1f6426ef18fb06e767ae874f1b88550eac3c546056d1c8d6924d7620-py3-none-any.whl' and adding '/tmp/tmpmfu05amv' to it
adding 'chicago_taxi_transform.py'
adding 'taxi_constants.py'
adding 'taxi_trainer.py'
adding 'taxi_transform.py'
adding 'taxi_tuner.py'
adding 'tfx_user_code_Transform-0.0+73bae87b1f6426ef18fb06e767ae874f1b88550eac3c546056d1c8d6924d7620.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+73bae87b1f6426ef18fb06e767ae874f1b88550eac3c546056d1c8d6924d7620.

ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell executes the Transform component, processing the input data according to the defined transformations. The `enable_cache=False` parameter ensures that the Transform component runs even if it has been executed before, applying the latest transformations.</span>


In [33]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "day_cos_xf"
    value {
      float_list {
        value: 0.5166333317756653
      }
    }
  }
  feature {
    key: "day_sin_xf"
    value {
      float_list {
        value: -1.2929933071136475
      }
    }
  }
  feature {
    key: "demand"
    value {
      float_list {
        value: 1.6094379425048828
      }
    }
  }
  feature {
    key: "duration_xf"
    value {
      float_list {
        value: -0.9746989607810974
      }
    }
  }
  feature {
    key: "hour_cos_xf"
    value {
      float_list {
        value: 1.6667683124542236
      }
    }
  }
  feature {
    key: "hour_sin_xf"
    value {
      float_list {
        value: 0.1684337705373764
      }
    }
  }
  feature {
    key: "log_duration"
    value {
      float_list {
        value: 2.39486026763916
      }
    }
  }
  feature {
    key: "log_trip_miles"
    value {
      float_list {
        value: 1.8012971878051758
      }
    }
  }
  feature {
    key: "log_trip_total"
    value 

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell retrieves and prints the output of the Transform component. It's a step to ensure that the component has successfully executed and to make the output location readily available for subsequent steps in the pipeline.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Generating Post-Transformation Statistics</strong></h4>


In [34]:
# Create StatisticsGen for the transformed data
post_transform_statistics_gen = tfx.components.StatisticsGen(
    examples=transform.outputs['transformed_examples']
)

# Run the component
context.run(post_transform_statistics_gen)

# Visualize the statistics
context.show(post_transform_statistics_gen.outputs["statistics"])


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell creates a StatisticsGen component to generate statistics for the transformed data. It provides insights into the data post-transformation, which is important for validating the transformation logic and understanding the dataset's characteristics.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Defining Trainer Module File Name</strong></h4>


In [36]:
_taxi_trainer_module_file = 'taxi_trainer.py'

<h4 style='color:darkblue;margin-left:20px;'><strong>Fully connected neural network, also known as a Dense Neural Network</strong></h4>

In [74]:
%%writefile {_taxi_trainer_module_file}

from typing import NamedTuple, Dict, List, Text, Any
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tfx import v1 as tfx
from keras_tuner.engine import base_tuner
from keras_tuner import HyperParameters, RandomSearch,BayesianOptimization
from tfx_bsl.public import tfxio
from tfx.components.trainer.fn_args_utils import FnArgs
import os
import taxi_constants
_LABEL_KEY = taxi_constants.LABEL_KEY


_BATCH_SIZE = 64

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', BayesianOptimization), ('fit_kwargs', Dict[Text, Any])])  # Changed to BayesianOptimization


early_stopping = EarlyStopping(
    monitor='val_mean_absolute_error',
    patience=10,
    restore_best_weights=True
)

def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = _BATCH_SIZE) -> tf.data.Dataset:
    return data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(batch_size=batch_size, label_key=_LABEL_KEY),
        tf_transform_output.transformed_metadata.schema).repeat()


def _build_keras_model(hp, tf_transform_output: tft.TFTransformOutput) -> tf.keras.Model:
    # Define feature specs and create input layers
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    # Remove the label feature
    feature_spec.pop(_LABEL_KEY)
    inputs = {
        key: tf.keras.layers.Input(shape=(1,), name=key)
        for key in feature_spec.keys()
    }

    # Concatenate all input features
    concatenated_inputs = tf.keras.layers.Concatenate()(list(inputs.values()))

    num_layers = hp.Int('num_layers', 1, 5)
    activation_choice = hp.Choice('activation', ['relu', 'leaky_relu', 'elu', 'tanh', 'sigmoid'])

    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=32, max_value=512, step=32)
        concatenated_inputs = tf.keras.layers.Dense(
            units=units,
            activation=activation_choice,
            kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2_{i}', 1e-5, 1e-2, sampling='log'))
        )(concatenated_inputs)
        if hp.Boolean(f'dropout_{i}'):
            dropout_rate = hp.Float(f'dropout_rate_{i}', 0.1, 0.5)
            concatenated_inputs = tf.keras.layers.Dropout(dropout_rate)(concatenated_inputs)

    # Output layer for regression
    output = tf.keras.layers.Dense(1, activation='linear')(concatenated_inputs)

    # Create and compile the Keras model
    model = tf.keras.Model(inputs=inputs, outputs=output)
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='mean_squared_error',
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )
    model.summary()
    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    tuner = BayesianOptimization(
        hypermodel=lambda hp: _build_keras_model(hp, tf_transform_output),
        objective='val_mean_absolute_error',
        max_trials=25,
        executions_per_trial=1,
        directory=fn_args.working_dir,
        project_name='taxi_trips_tuning_bayesian_optimization'
    )

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, tf_transform_output, _BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, tf_transform_output, _BATCH_SIZE)

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': 1500,  
            'validation_steps': 969,  
            'callbacks': [early_stopping]
        }
    )



def _get_tf_examples_serving_signature(model, tf_transform_output):
    model.tft_layer_inference = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def serve_tf_examples_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_inference(raw_features)
        model_input_keys = [layer.name for layer in model.layers if isinstance(layer, tf.keras.layers.InputLayer)]
        filtered_features = {key: value for key, value in transformed_features.items() if key in model_input_keys}
        outputs = model(filtered_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn



def _get_transform_features_signature(model, tf_transform_output):
    model.tft_layer_eval = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def transform_features_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_eval(raw_features)
        return transformed_features

    return transform_features_fn

def export_serving_model(tf_transform_output, model, output_dir):
    model.tft_layer = tf_transform_output.transform_features_layer()
    signatures = {
        'serving_default': _get_tf_examples_serving_signature(model, tf_transform_output),
        'transform_features': _get_transform_features_signature(model, tf_transform_output),
    }
    model.save(output_dir, save_format='tf', signatures=signatures)
    
def run_fn(fn_args: FnArgs):
    tensorboard_log_dir = os.path.join(fn_args.model_run_dir, 'logs')
    model_dir = os.path.join(fn_args.model_run_dir, 'model')
    os.makedirs(tensorboard_log_dir, exist_ok=True)

  
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(fn_args.model_run_dir, 'logs'),  
        update_freq='batch'
    )

    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_error', 
        mode='min', 
        patience=3,
        restore_best_weights=True
    )

    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, 'best_model'),
        monitor='val_mean_absolute_error', 
        mode='min', 
        save_best_only=True,
        verbose=1
    )

 
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Set up the tuner for hyperparameter tuning
    tuner_fn_result = tuner_fn(fn_args)
    tuner = tuner_fn_result.tuner

    # Create datasets for training and evaluation
    train_dataset = _input_fn(
        fn_args.train_files, 
        fn_args.data_accessor, 
        tf_transform_output, 
        _BATCH_SIZE
    )
    eval_dataset = _input_fn(
        fn_args.eval_files, 
        fn_args.data_accessor, 
        tf_transform_output, 
        _BATCH_SIZE
    )

    # Perform hyperparameter search using the tuner
    tuner.search(**tuner_fn_result.fit_kwargs)

    # Retrieve the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Build the best Keras model based on the best hyperparameters
    model = _build_keras_model(best_hps, tf_transform_output)

    # Determine the number of steps per epoch
    total_train_examples = fn_args.train_steps * _BATCH_SIZE
    steps_per_epoch = total_train_examples // _BATCH_SIZE

    # Fit the model with the callbacks
    model.fit(
        train_dataset,
        steps_per_epoch=steps_per_epoch,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        epochs=fn_args.train_steps // steps_per_epoch,
        callbacks=[
            tensorboard_callback,
            early_stopping_callback,
            model_checkpoint_callback
        ]
    )
    signatures = {
      'serving_default': _get_tf_examples_serving_signature(model, tf_transform_output),
    }
    
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)
    # Export the serving model
    export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)



Overwriting taxi_trainer.py


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell defines and writes the module file for training the model. It includes functions for input data processing, model architecture definition, hyperparameter tuning using Keras Tuner, and model training. This modular approach organizes the training logic and makes it reusable and maintainable.</span>



<span style='color:darkblue'><strong>Model Evaluation Metric Implementation:</strong></span>
<span style='color:black'>
The primary evaluation metric for our model is <em style='color:green;'><strong>Mean Absolute Error (MAE)</strong></em>, accompanied by <em style='color:orange;'><strong>Root Mean Squared Error (RMSE)</strong></em> as a secondary metric. 

- <strong>Why MAE is Optimal:</strong> MAE aligns well with our business objectives, offering a clear average error in the same units as the predicted variable, which is essential for stakeholder communication. Its robustness to outliers ensures a more stable and reliable performance metric, avoiding undue influence from data anomalies. Additionally, MAE's simplicity aids in its interpretability, particularly valuable in business environments.

- <strong>RMSE: Secondary Metric</strong>: RMSE complements MAE by providing insights into the impact of larger errors on the model's performance. Its higher sensitivity to outliers is beneficial in scenarios where large errors need significant attention. Using RMSE alongside MAE enables a more detailed analysis of the model's accuracy, highlighting the influence of larger prediction errors.
</span>



<h4 style='color:darkblue;margin-left:20px;'><strong>Hyperparameter Tuner</strong></h4>


In [101]:
from tfx.components import Tuner
from keras_tuner.engine import base_tuner

from tfx.proto import trainer_pb2
# from keras_tuner.engine import base_tuner
# Define the Tuner component
tuner = Tuner(
    module_file=_taxi_trainer_module_file,  # Path to Python file defining tuner_fn
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=10160),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=5716) 
)


In [102]:
# Add the Tuner component to the pipeline
context.run(tuner, enable_cache=False)

Trial 15 Complete [00h 00m 22s]
val_mean_absolute_error: 0.7854061126708984

Best val_mean_absolute_error So Far: 0.7578417062759399
Total elapsed time: 00h 03m 31s
Results summary
Results in /tmp/tfx-interactive-2023-12-03T22_08_59.188858-n_5s3bqq/.temp/30/taxi_trips_tuning_bayesian_optimization
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 06 summary
Hyperparameters:
num_layers: 2
activation: leaky_relu
units_0: 224
l2_{i}: 0.0002327414880974826
dropout_0: False
learning_rate: 0.0029623239448284167
dropout_rate_0: 0.4061595520633605
units_1: 224
dropout_1: True
units_2: 416
dropout_2: True
units_3: 96
dropout_3: False
units_4: 64
dropout_4: True
dropout_rate_1: 0.33846439332566014
dropout_rate_2: 0.4006153511209005
dropout_rate_3: 0.155354875403606
Score: 0.7578417062759399

Trial 03 summary
Hyperparameters:
num_layers: 3
activation: leaky_relu
units_0: 64
l2_{i}: 6.224065081134312e-05
dropout_0: False
learning_rate: 0.005441667012386122
dro

ExecutionResult(
    component_id: Tuner
    execution_id: 30
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>Here, the Tuner component is executed, which performs hyperparameter tuning based on the defined tuner function. By setting `enable_cache=False`, it ensures that the tuner runs irrespective of previous executions, facilitating fresh tuning runs.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Trainer </strong></h4>


In [103]:
from tfx.components import Trainer
import os
from tfx.proto import trainer_pb2
trainer = Trainer(
    module_file=_taxi_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=10160),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=5716) 
)

In [104]:
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying chicago_taxi_transform.py -> build/lib
copying taxi_transform.py -> build/lib
copying taxi_trainer.py -> build/lib
copying taxi_constants.py -> build/lib
copying taxi_tuner.py -> build/lib
installing to /tmp/tmp5_q002mx
running install
running install_lib
copying build/lib/taxi_transform.py -> /tmp/tmp5_q002mx
copying build/lib/taxi_tuner.py -> /tmp/tmp5_q002mx
copying build/lib/taxi_constants.py -> /tmp/tmp5_q002mx
copying build/lib/taxi_trainer.py -> /tmp/tmp5_q002mx
copying build/lib/chicago_taxi_transform.py -> /tmp/tmp5_q002mx
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp5_q002mx/tfx_user_code_Trainer-0.0+2684423181db0c79c6494cc4a6908bb7f122f56ad549bf0c5d15a0e94c8354ea-py3.10.egg-info
running install_scripts
creating /tmp/tmp5_q002mx/tfx_user_code_Trainer-0.0+2684423181db0c79c6494cc4a6908bb7f122f56ad549bf0c5d15a0e94c8354ea.dist-info/WHEEL
creating '/tmp/tmpgcik3343/tfx_user_code_Trainer-0.0+2684423181db0c79c6494cc4a6908bb7f122f56ad549bf0c5d15a0e94c8354ea-py3-none-any.whl' and adding '/tmp/tmp5_q002mx' to it
adding 'chicago_taxi_transform.py'
adding 'taxi_constants.py'
adding 'taxi_trainer.py'
adding 'taxi_transform.py'
adding 'taxi_tuner.py'
adding 'tfx_user_code_Trainer-0.0+2684423181db0c79c6494cc4a6908bb7f122f56ad549bf0c5d15a0e94c8354ea.dist-info/METADATA'
adding 'tfx_user_code_Trainer-0.0+2684423181db0c79c6494cc4a6908bb7f122f56ad549bf0c5d15a0e94c8354ea.dist-info/WHEEL'

ExecutionResult(
    component_id: Trainer
    execution_id: 31
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell executes the Trainer component, triggering the model training process. It uses the previously set up training and evaluation data, model architecture, and hyperparameters to train the model.</span>

<span style='color:darkblue'><strong>Trade-offs Influencing Model Architecture:</strong></span>
<span style='color:black'>
As part of our strategy to optimize the model for taxi demand prediction, several critical trade-offs were considered in its architecture:

- <strong>Layer Configuration and Activation Functions:</strong> The model's structure, including the number of layers, the number of units in each layer, and the types of activation functions used, was fine-tuned through Bayesian Optimization. This process aimed to strike a balance between underfitting (high bias) and overfitting (high variance).

- <strong>Regularization and Dropout:</strong> To combat overfitting, we employed L2 regularization and dropout techniques. L2 regularization adds a penalty to the loss function, which increases with the complexity of the model. Dropout, on the other hand, randomly deactivates a fraction of neurons in a layer during training, reducing the model's reliance on any specific set of neurons and thus its variance.

- <strong>Hyperparameter Tuning:</strong> The Keras Tuner was utilized to systematically explore and evaluate various model configurations. This approach allowed us to balance the model's complexity, which could potentially increase variance, with its simplicity, which might otherwise lead to increased bias.

Overall, these architectural decisions were made to ensure that the model aligns with our business goals while effectively managing the bias-variance tradeoff. This careful tuning and consideration of the model's parameters and structure, coupled with the use of MAE as the primary evaluation metric, ensure that the model's performance is both robust and easily interpretable in a business context.
</span>



DNN Results

In [118]:
import math

log_mae = 0.7802
log_rmse = 1.1557

exp_mae = math.exp(log_mae)
exp_rmse = math.exp(log_rmse)

print(exp_mae, exp_rmse)


2.181908603583655 3.1762460117370868


In [44]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['variables', 'saved_model.pb', 'assets', 'keras_metadata.pb', 'fingerprint.pb']


<h4 style='color:darkblue;margin-left:20px;'><strong>Model Evaluation</strong></h4>


In [88]:
import taxi_constants
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(signature_name='serving_default', 
                                label_key=taxi_constants.LABEL_KEY)],
    slicing_specs=[tfma.SlicingSpec()], 
     metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='MeanAbsoluteError'),
                tfma.MetricConfig(
                    class_name='MeanAbsoluteError',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.9159}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.LOWER_IS_BETTER,
                            absolute={'value': 0.02}
                        )
                    )
                )
            ]
        )
    ]
)


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell sets up TensorFlow Model Analysis (TFMA) evaluation configurations. It defines the model specifications, slicing specifications, and metrics specifications, including the Mean Absolute Error and its threshold. This configuration is crucial for evaluating the model's performance and ensuring it meets specific criteria.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Initializing Model Resolver and Evaluator Components</strong></h4>


In [89]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

context.run(model_resolver)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    example_splits=['eval'],  # Use the evaluation split for evaluation
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 28
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>In this cell, a Resolver component is used to identify the latest blessed model, and an Evaluator component is initialized. The Evaluator performs model evaluation, comparing the new model against a baseline model (if available) to determine if it should be blessed for deployment.</span>

<span style='color:darkblue'><strong>Determining Bias/Variance and Architecture Optimization:</strong></span>
<span style='color:black'>
To optimize the model architecture for taxi demand prediction, we've focused on the critical balance between bias and variance, a fundamental aspect of machine learning model performance:

- <strong>Bias and Variance Analysis:</strong> The optimal model achieves a delicate balance between bias (error due to overly simplistic assumptions) and variance (error due to overfitting to the training data). Our trials have been geared towards finding this balance. For instance, in Trial 06, we observed a validation MAE of 0.7578, suggesting a successful balance achieved by this specific combination of layers, units, and regularization strategies. This trial possibly represents the most effective architecture in terms of bias-variance tradeoff.

- <strong>Early Stopping:</strong> To further refine our model and mitigate overfitting, we've implemented an EarlyStopping callback. This monitors the 'val_mean_absolute_error' during training. The training process is halted when there's no improvement in the validation metric, thereby preventing the model from learning noise and irrelevant patterns in the training data, a common cause of high variance.

These strategies and analytical approaches have been instrumental in fine-tuning the model's architecture, ensuring it not only aligns with our predictive goals but also maintains a robust performance by balancing the intricacies of bias and variance.
</span>


In [90]:
evaluator.outputs

{'evaluation': OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None,
 'blessing': OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None}

In [91]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Dec  4 00:37 BLESSED


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>Here, the cell lists the contents of the directory where the Evaluator's blessing output is stored. This output indicates whether the current model passed the evaluation criteria and is "blessed" for potential deployment.</span>



In [92]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}



<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell fetches and displays the TensorFlow Model Analysis (TFMA) evaluation results. These results include detailed metrics and analyses that are critical for assessing the model's performance.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Initializing and Running Pusher Component</strong></h4>


In [94]:
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
pusher = tfx.components.Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
      push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=SERVING_MODEL_DIR
        )
    ),
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 29
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell sets up and executes the Pusher component of TFX. The Pusher is responsible for deploying the trained model to a specified destination, conditional on the model passing the evaluation stage. It uses the model artifact and model blessing from the Evaluator component to decide whether to push the model to the serving directory.</span>


In [109]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Dec  4 00:37 BLESSED


<h4 style='color:darkblue;margin-left:20px;'><strong>TFX Pipeline </strong></h4>


In [114]:
import os
import tensorflow_model_analysis as tfma
import tfx
from tfx import v1 as tfx
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Pusher,Evaluator
from tfx.orchestration import pipeline
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
import taxi_constants

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str, module_file: str, serving_model_dir: str) -> tfx.dsl.Pipeline:
    metadata_path = os.path.join(pipeline_root, 'metadata.db')
    os.makedirs(os.path.dirname(metadata_path), exist_ok=True)
    
    metadata_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path)
    input_config = example_gen_pb2.Input(splits=[
        example_gen_pb2.Input.Split(name='train', pattern=TRAIN_DATA_FILENAME),
        example_gen_pb2.Input.Split(name='eval', pattern=VALIDATION_DATA_FILENAME),
        example_gen_pb2.Input.Split(name='test', pattern= TEST_DATA_FILENAME)
    ])
    example_gen = CsvExampleGen(input_base=data_root, input_config=input_config)
    statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
    schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )
    transform = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(TRANSFORM_MODULE_PATH)
    )
    tuner = Tuner(
        module_file=_taxi_trainer_module_file,
        examples=transform.outputs['transformed_examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=10160),
        eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=5716)
    )
    trainer = Trainer(
        module_file=os.path.abspath(module_file),
        transformed_examples=transform.outputs['transformed_examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        hyperparameters=tuner.outputs['best_hyperparameters'],
        train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=10160),
        eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=5716)
    )
    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(signature_name='serving_default', label_key=taxi_constants.LABEL_KEY)],
        slicing_specs=[tfma.SlicingSpec()], 
        metrics_specs=[
            tfma.MetricsSpec(
                metrics=[
                    tfma.MetricConfig(class_name='MeanAbsoluteError'),
                    tfma.MetricConfig(
                        class_name='MeanAbsoluteError',
                        threshold=tfma.MetricThreshold(
                            value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.8159}),
                            change_threshold=tfma.GenericChangeThreshold(
                                direction=tfma.MetricDirection.LOWER_IS_BETTER,
                                absolute={'value': 0.02}
                            )
                        )
                    )
                ]
            )
        ]
    )
    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing)
    ).with_id('latest_blessed_model_resolver')
    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config
    )
    pusher = Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=serving_model_dir
            )
        )
    )
    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform, 
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher
    ]
    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        metadata_connection_config=metadata_config
    )


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>This cell defines a function to create a TFX pipeline. The function includes setting up all the necessary components of the pipeline, such as ExampleGen, StatisticsGen, SchemaGen, and others, configuring them with appropriate inputs and parameters. This function is a key part of orchestrating the entire ML workflow.</span>


<h4 style='color:darkblue;margin-left:20px;'><strong>Executing Local TFX Pipeline</strong></h4>


In [115]:
import tfx
from tfx import v1 as tfx
import kfp
from tfx.orchestration.metadata import sqlite_metadata_connection_config
tfx.orchestration.LocalDagRunner().run(
      _create_pipeline(
          pipeline_name=PIPELINE_NAME,
          pipeline_root=PIPELINE_ROOT,
          data_root=DATA_DIRECTORY,
          module_file=_taxi_trainer_module_file,
          serving_model_dir=SERVING_MODEL_DIR
      ))

Trial 15 Complete [00h 00m 22s]
val_mean_absolute_error: 0.8473232388496399

Best val_mean_absolute_error So Far: 0.7802023887634277
Total elapsed time: 00h 04m 56s
Results summary
Results in gs://chicago_taxitrips/pipeline_root/demand-prediction/Tuner/.system/executor_execution/76/.temp/76/taxi_trips_tuning_bayesian_optimization
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 11 summary
Hyperparameters:
num_layers: 5
activation: leaky_relu
units_0: 288
l2_{i}: 0.006591216764804274
dropout_0: False
learning_rate: 0.00016465304756071368
units_1: 416
dropout_1: True
units_2: 288
dropout_2: True
units_3: 96
dropout_3: False
dropout_rate_0: 0.45432170066215816
dropout_rate_1: 0.11544737164337643
units_4: 160
dropout_4: True
dropout_rate_2: 0.3454379152132875
dropout_rate_3: 0.2770758634304471
dropout_rate_4: 0.378230243413198
Score: 0.7802023887634277

Trial 03 summary
Hyperparameters:
num_layers: 2
activation: relu
units_0: 256
l2_{i}: 0.0015778424

Processing /tmp/tmp79195iqd/tfx_user_code_Trainer-0.0+2684423181db0c79c6494cc4a6908bb7f122f56ad549bf0c5d15a0e94c8354ea-py3-none-any.whl
Reloading Tuner from ./taxi_trips_tuning_bayesian_optimization/tuner0.json
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 day_cos_xf (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 day_sin_xf (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 duration_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                               

2023-12-04 03:12:14.979621: I tensorflow/tsl/platform/cloud/gcs_file_system.cc:627] ### RequestUploadSessionStatus: completed = 1, already_uploaded = 0, file = gs://chicago_taxitrips/pipeline_root/demand-prediction/Trainer/model_run/77/logs/train/events.out.tfevents.1701659529.e28681d333e4.1.10.v2


   37/10160 [..............................] - ETA: 24:14 - loss: 1.0473 - mean_absolute_error: 0.7375 - root_mean_squared_error: 1.0091

2023-12-04 03:12:16.766647: I tensorflow/tsl/platform/cloud/gcs_file_system.cc:627] ### RequestUploadSessionStatus: completed = 1, already_uploaded = 0, file = gs://chicago_taxitrips/pipeline_root/demand-prediction/Trainer/model_run/77/logs/train/events.out.tfevents.1701659529.e28681d333e4.1.10.v2


...

10124/10160 [============================>.] - ETA: 5s - loss: 0.9445 - mean_absolute_error: 0.7026 - root_mean_squared_error: 0.9701

2023-12-04 03:39:50.060711: I tensorflow/tsl/platform/cloud/gcs_file_system.cc:627] ### RequestUploadSessionStatus: completed = 1, already_uploaded = 0, file = gs://chicago_taxitrips/pipeline_root/demand-prediction/Trainer/model_run/77/logs/train/events.out.tfevents.1701659529.e28681d333e4.1.10.v2


10153/10160 [============================>.] - ETA: 1s - loss: 0.9448 - mean_absolute_error: 0.7027 - root_mean_squared_error: 0.9702

2023-12-04 03:39:52.968014: I tensorflow/tsl/platform/cloud/gcs_file_system.cc:627] ### RequestUploadSessionStatus: completed = 1, already_uploaded = 0, file = gs://chicago_taxitrips/pipeline_root/demand-prediction/Trainer/model_run/77/logs/train/events.out.tfevents.1701659529.e28681d333e4.1.10.v2


10157/10160 [============================>.] - ETA: 0s - loss: 0.9448 - mean_absolute_error: 0.7027 - root_mean_squared_error: 0.9702
Epoch 1: val_mean_absolute_error improved from inf to 0.81526, saving model to gs://chicago_taxitrips/pipeline_root/demand-prediction/Trainer/model_run/77/model/best_model
10160/10160 [==============================] - 1691s 166ms/step - loss: 0.9448 - mean_absolute_error: 0.7027 - root_mean_squared_error: 0.9703 - val_loss: 1.3525 - val_mean_absolute_error: 0.8153 - val_root_mean_squared_error: 1.1628


<span style='color:darkblue'><strong>Comment:</strong></span> <span style='color:black'>In this cell, the TFX pipeline, as defined in the previous cell, is executed locally using TFX's LocalDagRunner. This step initiates the entire pipeline process, running each component in the defined sequence and processing the data through the stages of the ML workflow.</span>


---
<h3 style='color:darkblue;text-align:center;'><strong>Conclusion and Future Directions</strong></h3>
<p style='color:black; font-size:16px; text-align:justify;'>
As our Interactive TFX Pipeline for Taxi Demand Prediction comes to a close, we celebrate our significant achievements. Our model, a complex Multi-Layer Perceptron, has proven effective in forecasting taxi demand, indicated by a <span style='color:green;'><strong>Mean Absolute Error (MAE) of 2.18</strong></span> and a <span style='color:green;'><strong>Root Mean Squared Error (RMSE) of 3.19</strong></span>. To put this in simpler terms, imagine you're guessing the number of taxi trips in Chicago each hour. On average, your guess would be off by about 2 to 3 trips. For instance, if there were actually 20 trips in an hour, the model's prediction would likely range between 17 and 23 trips.

This level of accuracy demonstrates a well-managed balance between bias and variance, a crucial aspect in predictive modeling. Our model avoids overfitting (capturing random noise in the data) and underfitting (failing to capture underlying patterns), ensuring predictions that are both reliable and adaptable to real-world scenarios.

The next stage involves deploying and running this pipeline using Kubeflow orchestration on Vertex AI.This move is aimed at making our solution production-ready,Employing Kubeflow for orchestration provides us with a robust, scalable platform that facilitates the management of our TFX pipeline workflows in a more dynamic and efficient manner. This setup is especially advantageous for handling complex machine learning workflows and will play a pivotal role in ensuring that our model adapts to varying operational loads and data volumes seamlessly.

</p>


---